In [1]:
# === Imports ===
import os
import numpy as np
import pandas as pd
from glob import glob
from tqdm.notebook import tqdm
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.preprocessing import MinMaxScaler
import torch.serialization
torch.serialization.add_safe_globals([MinMaxScaler])
import matplotlib.pyplot as plt

In [2]:
#Testing config
config = {
    "model_type": "lstm", # Options: "lstm", "cnn", "tcn", "transformer" 
    "seed": 42,
    "input_size": 6,
    "output_size": 3,
    "train_split": 0.8,
    "num_epochs": 50,
    "learning_rate": 0.001,
    "batch_size": 64,
    "hidden_size": 128,
    "num_layers": 4,
    "dropout": 0.1,
    "seq_len": 150,
    "patience": 10,
    "bidirectional": False,
    "device": "cuda" if torch.cuda.is_available() else "cpu",
    "train_data_path": "./data/testing_layouts/", #"./data/extracted_track_data/",
    "test_data_path": "./data/testing_layouts/",
    "model_save_path": "./models/t_racing_line_lstm.pt",
    "input_cols": ["left_x","left_y","left_z","right_x","right_y","right_z"],
    "output_cols": ["x","y","z"]
}

In [3]:
# === Model with Attention ===
class RacingLineLSTMWithAttention(nn.Module):
    def __init__(self, config, scaler_x=None, scaler_y=None):
        super().__init__()
        self.bidirectional = config["bidirectional"]
        self.hidden_size = config["hidden_size"]
        self.num_directions = 2 if self.bidirectional else 1

        self.lstm = nn.LSTM(
            input_size=config["input_size"],
            hidden_size=self.hidden_size,
            num_layers=config["num_layers"],
            dropout=config["dropout"],
            batch_first=True,
            bidirectional=self.bidirectional
        )

        self.attn = nn.Linear(self.num_directions * self.hidden_size, 1)
        self.dropout = nn.Dropout(config["dropout"])
        self.fc = nn.Linear(self.num_directions * self.hidden_size, config["output_size"])

        # Optional scalers for external inference use
        self.scaler_x = scaler_x
        self.scaler_y = scaler_y

    def forward(self, x):
        lstm_out, _ = self.lstm(x) # x: (batch, seq_len, input_size)
        attn_scores = self.attn(lstm_out)
        attn_weights = torch.softmax(attn_scores, dim=1)
        context = torch.sum(attn_weights * lstm_out, dim=1)
        context = self.dropout(context)
        return self.fc(context)

    def get_attention_weights(self, x):
        """Optional: for visualization/debugging"""
        lstm_out, _ = self.lstm(x)
        attn_scores = self.attn(lstm_out)
        attn_weights = torch.softmax(attn_scores, dim=1)
        return attn_weights.squeeze(-1)  # (batch, seq_len)
    

In [4]:
# Some other basic models to try (needs testing)

#=======================================================================================
# 1D CNN
class RacingLineCNN(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(config["input_size"], 64, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.Conv1d(64, 128, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.AdaptiveAvgPool1d(1),
            nn.Flatten(),
            nn.Linear(128, config["output_size"])
        )

    def forward(self, x):
        x = x.permute(0, 2, 1)  # CNN expects [Batch size, Channels (features), Time (sequence length)]
        return self.net(x)

#=======================================================================================
# Temporal Convolutional Network
class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super().__init__()
        self.chomp_size = chomp_size

    def forward(self, x):
        return x[:, :, :-self.chomp_size]

class TemporalBlock(nn.Module):
    def __init__(self, in_ch, out_ch, kernel_size, stride, dilation, padding, dropout):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv1d(in_ch, out_ch, kernel_size, stride, padding, dilation=dilation),
            Chomp1d(padding),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Conv1d(out_ch, out_ch, kernel_size, stride, padding, dilation=dilation),
            Chomp1d(padding),
            nn.ReLU(),
            nn.Dropout(dropout)
        )
        self.downsample = nn.Conv1d(in_ch, out_ch, 1) if in_ch != out_ch else None

    def forward(self, x):
        out = self.block(x)
        res = x if self.downsample is None else self.downsample(x)
        return out + res

class RacingLineTCN(nn.Module):
    def __init__(self, config, levels=2, kernel_size=2):
        super().__init__()
        layers = []
        in_ch = config["input_size"]
        for i in range(levels):
            out_ch = 64
            dilation = 2 ** i
            padding = (kernel_size - 1) * dilation
            layers.append(TemporalBlock(in_ch, out_ch, kernel_size, 1, dilation, padding, config["dropout"]))
            in_ch = out_ch
        self.network = nn.Sequential(*layers)
        self.linear = nn.Linear(in_ch, config["output_size"])

    def forward(self, x):
        x = x.permute(0, 2, 1)
        out = self.network(x)
        out = out[:, :, -1]
        return self.linear(out)

#=======================================================================================
# Transformer Model
class LearnablePositionalEncoding(nn.Module):
    def __init__(self, max_len, d_model):
        super().__init__()
        self.pos_embedding = nn.Parameter(torch.randn(1, max_len, d_model))

    def forward(self, x):
        return x + self.pos_embedding[:, :x.size(1), :]

class RacingLineTransformer(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.embedding = nn.Linear(config["input_size"], config["hidden_size"])
        self.pos_encoding = LearnablePositionalEncoding(max_len=config["seq_len"], d_model=config["hidden_size"])
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=config["hidden_size"],
            nhead=4,
            dropout=config["dropout"],
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=config["num_layers"])
        self.fc = nn.Linear(config["hidden_size"], config["output_size"])

    def forward(self, x):
        x = self.embedding(x)
        x = self.pos_encoding(x) 
        x = self.transformer(x)
        return self.fc(x)
    
#=======================================================================================
# GNN/GCN (an idea to try if we desire)



In [5]:
def get_model(config, scaler_x=None, scaler_y=None):
    match config["model_type"]:
        case "lstm":
            return RacingLineLSTMWithAttention(config, scaler_x=scaler_x, scaler_y=scaler_y)
        case "cnn":
            return RacingLineCNN(config)
        case "tcn":
            return RacingLineTCN(config)
        case "transformer":
            return RacingLineTransformer(config)
        case _:
            raise ValueError(f"Unknown model type: {config['model_type']}")

def load_model(path, config):
    checkpoint = torch.load(path, map_location=config["device"], weights_only=False)
    cfg = checkpoint["config"]
    model = get_model(cfg)
    model.load_state_dict(checkpoint["model_state_dict"])
    model.to(cfg["device"])
    scaler_x = checkpoint["scaler_x"]
    scaler_y = checkpoint["scaler_y"]
    return model, scaler_x, scaler_y

In [6]:
def print_feature_accuracy(preds, trues, scaler_y, feature_names):
    preds = np.array(preds)
    trues = np.array(trues)

    print(f"\nPer-Feature Accuracy (%):")
    print("-" * 60)
    for i, name in enumerate(feature_names):
        range_train = scaler_y.scale_[i]
        range_test = trues[:, i].max() - trues[:, i].min()

        if range_test == 0:
            print(f"{name:>16}: N/A (zero test range)")
            continue

        mean_error = np.mean(np.abs(preds[:, i] - trues[:, i]))
        acc_train = (1 - (mean_error / range_train)) * 100
        acc_test = (1 - (mean_error / range_test)) * 100

        acc_train = max(0.0, min(100.0, acc_train))
        acc_test = max(0.0, min(100.0, acc_test))

        print(f"{name:>16}: {acc_test:6.2f}% (layout-based)   {acc_train:6.2f}% (train-scale)")

# === Inference for Sequence-to-Sequence on Non-Circular Tracks ===
def run_inference(config, data_folder, model_path):
    print("Loading model and scalers...")
    model, scaler_x, scaler_y = load_model(model_path, config)
    model.eval()

    print("Loading unseen layouts from:", data_folder)
    layout_files = sorted(glob(os.path.join(data_folder, "*.csv")))
    total_layouts = len(layout_files)
    print(f"Found {total_layouts} layout files.\n")

    for layout_index, layout_path in enumerate(layout_files):
        layout_name = os.path.basename(layout_path)
        print(f"[{layout_index + 1}/{total_layouts}] Predicting layout: {layout_name}")

        df = pd.read_csv(layout_path)
        X = df[config["input_cols"]].values
        Y = df[config["output_cols"]].values
        X_scaled = scaler_x.transform(X)
        n = len(X_scaled)

        trues_real = Y.copy()
        preds_real = np.zeros_like(Y)
        count_map = np.zeros((n, 1))

        for i in tqdm(range(0, n - config["seq_len"]+1), desc=f"[{layout_index + 1}/{total_layouts}]"):
            seq = X_scaled[i:i + config["seq_len"]] 
            X_tensor = torch.tensor(seq.reshape(1, config["seq_len"], -1), dtype=torch.float32).to(config["device"])

            with torch.no_grad():
                pred_scaled = model(X_tensor).cpu().squeeze(0).numpy()  
                pred_real = scaler_y.inverse_transform(pred_scaled.reshape(1, -1))[0] 
                # print("pred_real shape:", pred_real.shape)
                # print("pred_real[:5]:", pred_real[:5])
                # print("pred_scaled shape:", pred_scaled.shape)
                # print("pred_scaled[:5]:", pred_scaled[:5])

            for j in range(config["seq_len"]):
                idx = i + j
                if idx < n:
                    preds_real[idx] += pred_real[j]
                    count_map[idx] += 1

        count_map[count_map == 0] = 1
        preds_real /= count_map

        # Plot
        plt.figure(figsize=(12, 6))
        plt.plot(trues_real[:, 0], trues_real[:, 2], label="True", linewidth=2)
        plt.plot(preds_real[:, 0], preds_real[:, 2], label="Predicted", linewidth=2, linestyle="--")
        plt.title(f"X vs Z: {layout_name}")
        plt.xlabel("X Coordinate")
        plt.ylabel("Z Coordinate")
        plt.axis("equal")
        plt.grid(True)
        plt.legend()
        plt.show()

        # Accuracy
        print_feature_accuracy(preds_real, trues_real, scaler_y, config["output_cols"])

        # Spatial Error
        spatial_errors = np.linalg.norm(preds_real[:, [0, 2]] - trues_real[:, [0, 2]], axis=1)
        mean_spatial_error = np.mean(spatial_errors)
        max_spatial_error = np.max(spatial_errors)
        print(f"Mean X/Z spatial error: {mean_spatial_error:.2f}m, Max: {max_spatial_error:.2f}m\n")

# === Run it ===
run_inference(config, data_folder="./data/testing_layouts", model_path="./models/t_racing_line_lstm.pt")


Loading model and scalers...
Loading unseen layouts from: ./data/testing_layouts
Found 1 layout files.

[1/1] Predicting layout: ks_barcelona_layout_gp_Processed_Data.csv


[1/1]:   0%|          | 0/2779 [00:00<?, ?it/s]

IndexError: index 3 is out of bounds for axis 0 with size 3